In [46]:
# imports:
import pandas as pd
from sklearn.cluster import KMeans
import pickle
pazak = {0:'Pazak',1:'Chatlanin'}

In [8]:
# Load euclidean distances data
df = pd.read_csv('Data/faces_euclidean_features.csv')
df.head()

,left_eye-right_eye,left_eye-nose,left_eye-mouth_left,left_eye-mouth_right,right_eye-left_eye,right_eye-nose,right_eye-mouth_left,right_eye-mouth_right,nose-left_eye,nose-right_eye,nose-mouth_left,nose-mouth_right,mouth_left-left_eye,mouth_left-right_eye,mouth_left-nose,mouth_left-mouth_right,mouth_right-left_eye,mouth_right-right_eye,mouth_right-nose,mouth_right-mouth_left
0,0.436686,0.293929,0.395205,0.534884,0.436686,0.414687,0.623971,0.396065,0.293929,0.414687,0.209624,0.274371,0.395205,0.623971,0.209624,0.414936,0.534884,0.396065,0.274371,0.414936
1,0.456806,0.315478,0.393492,0.565061,0.456806,0.306828,0.578675,0.385199,0.315478,0.306828,0.271917,0.249626,0.393492,0.578675,0.271917,0.384240,0.565061,0.385199,0.249626,0.384240
2,0.322055,0.361824,0.398738,0.552498,0.322055,0.132640,0.432794,0.386573,0.361824,0.132640,0.332479,0.254903,0.398738,0.432794,0.332479,0.285892,0.552498,0.386573,0.254903,0.285892
3,0.460852,0.304153,0.346807,0.524620,0.460852,0.320136,0.588235,0.354559,0.304153,0.320136,0.270767,0.221016,0.346807,0.588235,0.270767,0.407139,0.524620,0.354559,0.221016,0.407139
4,0.458330,0.338282,0.357675,0.570839,0.458330,0.268791,0.537670,0.352675,0.338282,0.268791,0.271729,0.233396,0.357675,0.537670,0.271729,0.395606,0.570839,0.352675,0.233396,0.395606


In [42]:
model = KMeans(n_clusters=2, random_state=42).fit(df.values)

In [47]:
with open("pazak.pkl", "wb") as f:
    pickle.dump(model, f)

In [ ]:
with open("model.pkl", "rb") as f:
    model = pickle.load(f)

In [54]:
pazak[model.predict([df.iloc[11]])[0]]

'Chatlanin'